In [2]:
"""
* Transformation explained.
*
* @author rambabu.posa
"""
from pyspark.sql import SparkSession
from pyspark.sql.functions import (lit,col,concat,expr)
import os

current_dir = os.path.dirname("")
relative_path = "../04 Laziness/data/NCHS_-_Teen_Birth_Rates_for_Age_Group_15-19_in_the_United_States_by_County.csv"
absolute_file_path = os.path.join(current_dir, relative_path)

# Step 1 - Creates a session on a local master
spark = SparkSession.builder.appName("Analysing Catalyst's behavior") \
                    .master("local[*]").getOrCreate()

# Step 2 - Reads a CSV file with header, stores it in a dataframe
df = spark.read.csv(header=True, inferSchema=True,path=absolute_file_path)

df0 = df

# Step 3 - Build a bigger dataset
df = df.union(df0)

# Step 4 - Cleanup. preparation
df = df.withColumnRenamed("Lower Confidence Limit", "lcl") \
       .withColumnRenamed("Upper Confidence Limit", "ucl")

# Step 5 - Transformation
df =  df.withColumn("avg", expr("(lcl+ucl)/2")) \
        .withColumn("lcl2", col("lcl")) \
        .withColumn("ucl2", col("ucl"))

# Step 6 - explain
df.explain()

spark.stop()

== Physical Plan ==
Union
:- *(1) Project [Year#10, State#11, County#12, State FIPS Code#13, County FIPS Code#14, Combined FIPS Code#15, Birth Rate#16, Lower Confidence Limit#17 AS lcl#37, Upper Confidence Limit#18 AS ucl#47, ((Lower Confidence Limit#17 + Upper Confidence Limit#18) / 2.0) AS avg#57, Lower Confidence Limit#17 AS lcl2#68, Upper Confidence Limit#18 AS ucl2#80]
:  +- *(1) FileScan csv [Year#10,State#11,County#12,State FIPS Code#13,County FIPS Code#14,Combined FIPS Code#15,Birth Rate#16,Lower Confidence Limit#17,Upper Confidence Limit#18] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/resources/spark_in_action/04 Laziness/data/NCHS_-_Teen_Birth_Rates_for_Ag..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Year:int,State:string,County:string,State FIPS Code:int,County FIPS Code:int,Combined FIPS...
+- *(2) Project [Year#10, State#11, County#12, State FIPS Code#13, County FIPS Code#14, Combined FIPS Code#15, Birth Rate#16, Lower Confidence Limit